# Deep Reinforcement Learning: 0 to 100

Using RL to teach robots to fly a drone

'https://towardsdatascience.com/deep-reinforcement-learning-for-dummies/'

<img src="Image/00/StartOfCourse00.jpg">

Ever wondered how you’d teach a robot to land a drone without programming every single move? That’s exactly what I set out to explore. I spent weeks building a game where a virtual drone has to figure out how to land on a platform—not by following pre-programmed instructions, but by learning from trial and error, just like how you learned to ride a bike.

This is Reinforcement Learning (RL), and it’s fundamentally different from other machine learning approaches. Instead of showing the AI thousands of examples of “correct” landings, you give it feedback: “Hey, that was pretty good, but maybe try being more gentle next time?” or “Yikes, you crashed—probably don’t do that again.” Through countless attempts, the AI figures out what works and what doesn’t.

In this post, I’m documenting my journey from RL basics to building a working system that (mostly!) teaches a drone to land. You’ll see the successes, the failures, and all the weird behaviors I had to debug along the way.

## 1. Reinforcement learning: Overview

A lot of the idea can be related to Pavlov’s dog and Skinner’s rat experiments. The idea is that you give the subject a ‘reward‘ when it does something you want it to do (positive reinforcement) and a ‘penalty‘ when it does something bad (negative reinforcement). Through many repeated attempts, your subject learns from this feedback, gradually discovering which actions lead to success—similar to how Skinner’s rat learned which lever presses produced food rewards.

<img src="Image/00/Pavlov.jpg">

In the same fashion, we want a system that will learn to do things (or tasks) such that it can maximize the reward and minimize the penalty. Note this fact about maximizing reward, which will come in later.

### 1.1 Core Concepts

When talking about systems that can be implemented programmatically on computers, the best practice is to write clear definitions for ideas that can be abstracted. In the study of AI (and more specifically, Reinforcement learning), the core ideas can be boiled down to the following:

1. **Agent** (or Actor): This is our subject from the previous section. This can be the dog, a robot trying to navigate a huge factory, a video game NPC, etc.
2. **Environment** (or the world): This can be a place, a simulation with restrictions, a video game’s virtual game world, etc. I think of this like, “A box, real or virtual, where the agent’s entire life is confined to; it only knows of what happens within the box. We, as the overlords, can alter this box, while the agent will think that god is exacting his will on his world.”
3. **Policy**: Just like in governments, companies, and many more similar entities, ‘policies’ dictate “What actions should be taken when given a certain situation”.
4. **State**: This is what the agent “sees” or “knows” about its current situation. Think of it as the agent’s snapshot of reality at any given moment—like how you see the traffic light color, your speed, and the distance to the intersection when driving.
5. **Action**: Now that our agent can ‘see’ things in its environment, it may want to do something about its state. Maybe it just woke up from a long night’s slumber, and now it wants to get a cup of coffee. In this case, the first thing it will do is get out of bed. This is an action that the agent will take to achieve its goal, i.e., GET SOME COFFEE!
6. **Reward**: Every time the actor executes an action (of its own volition), something may change in the world. For example, our agent got out of bed and started walking towards the kitchen, but then, because it is so bad at walking, it tripped and fell. In this situation, the god (us) rewards it with a punishment for being bad at walking (negative reward). But then the agent makes it to the kitchen and gets the coffee, so the god (us) rewards it with a cookie (positive reward).

<img src="Image/00/RL_illustration.jpg">

As you can imagine, most of these key components need to be tailored for the specific task/problem that we want the agent to solve.

## 2. The Gym

Now that we understand the basics, you might be wondering: **how do we actually build one of these systems?** Let me show you the game I built.

For this post, I have written a bespoke video game that anyone can access and use to train their own machine learning agent to play the game.

The full code repository can be found on GitHub (please star this). I intend to use this repository for more games and simulation code, along with more advanced techniques that I will implement in my next installments of posts on RL.

https://github.com/vedant-jumle/reinforcement-learning-101

### Delivery Drone

The delivery drone is a game where the objective is to fly a drone (likely containing deliveries) onto a platform. To win the game, we have to land. To land, we have to meet the following criteria:

1. Be in landing proximity to the platform
2. Be slow enough
3. Be upright (Landing upside down is more like crashing than landing)

All information on how to run the game can be found in the GitHub repository.

Here’s what the game looks like:

<img src="Image/00/Sample_game.jpg">

If the drone flies off the screen or touches the ground, it will be considered a ‘crash’ case and thus lead to a failure.

### State description

The drone observes 15 continuous values that completely describe its situation:

<img src="Image/00/WhtaDrone">

Landing Success Criteria: The drone must simultaneously achieve:

1. Horizontal alignment: within platform bounds (|dx| < 0.0625)
2. Safe approach speed: less than 0.3
3. Level orientation: tilt less than 20° (|angle| < 0.111)
4. Correct altitude: bottom of drone touching platform top

It’s like parallel parking—you need the right position, right angle, and moving slowly enough to not crash!

### How can someone design a policy?

There are many ways to design a policy. It can be Bayesian (maintaining probability distributions over beliefs), it can be a simple lookup table for discrete states, a hand-coded rule system (“if distance < 10, then brake”), a decision tree, or—as we’ll explore—a neural network that learns the mapping from states to actions through gradient descent.

Effectively, we want something that takes in the aforementioned state, performs some computation using this state, and returns what action should be performed.

### Deep Learning to build a policy?

So how do we design a policy that can handle continuous states (like exact drone positions) and learn complex behaviors? **This is where neural networks come in**.

In case of neural networks (or in deep learning), it is generally best to work with action probabilities, i.e., “What action is likely the best given the current state?”. So, we can define a neural network that will take in the state as a ‘vector’ or ‘collection of vectors’ as input. This vector or collection of vectors has to be constructed from the observed state. For our delivery drone game, the state vector is:

### State vector (from our 2D drone game)

The drone observes its absolute position, velocities, orientation, fuel, platform position, and derived metrics. Our continuous state is:

<img src="Image/00/StateVector2dDrone">

Where each component represents:

<img src="Image/00/Record1.webp">

All components are normalized to roughly [0,1] or [-1,1] ranges for stable neural network training.

### Action space (three independent binary thrusters)

Instead of discrete action combinations, we treat each thruster independently:

* Main thruster (upward thrust)
* Left thruster (clockwise rotation)
* Right thruster (counter-clockwise rotation)

Each action is sampled from a Bernoulli distribution, giving us 3 independent binary decisions per timestep.

### Neural-network policy (probabilistic with Bernoulli sampling)

Let fθ(s) be the network outputs after sigmoid activation. The policy uses independent Bernoulli distributions:

<img src="Image/00/Bernoulli.webp">